In [1]:
import os
import cv2
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed

import nibabel as nib
import h5py
import torch
import torch.nn.functional as F
from typing import *
from pathlib import Path

In [2]:
def read_and_preprocess_ct(file_path, metadata_row, hu_range=(-1000, 1000)):
    """
    Load and preprocess a CT scan:
    - Reorient to RAS (Right-Anterior-Superior) orientation
    - Apply DICOM rescale
    - Clip to [-1000, 1000] HU range
    """
    img = nib.load(str(file_path))
    
    # Reorient to RAS orientation (Right-Anterior-Superior)
    # This ensures consistent orientation: left->right, posterior->anterior, inferior->superior
    img = nib.as_closest_canonical(img)
    
    data = img.get_fdata()
    if metadata_row is not None:
        # Apply DICOM rescale if metadata is provided
        slope = float(metadata_row['RescaleSlope'])
        intercept = float(metadata_row['RescaleIntercept'])
        data = data * slope + intercept
    else:
        # Default rescale if no metadata is provided
        slope = 1.0
        intercept = 0.0
        data = data * slope + intercept
    data = np.clip(data, hu_range[0], hu_range[1])
    return data.astype(np.float32)


def resize_and_pad_3d_to_target(volume, target_shape=(160, 224, 224)):
    """
    Aspect-ratio-preserving resize to fit within target_shape, then center-pad to match it.
    Normalize HU values to [-1, 1] range, then convert to uint8 [0, 255].
    """
    d, h, w = volume.shape
    target_d, target_h, target_w = target_shape
    scale = min(target_d / d, target_h / h, target_w / w)
    new_d, new_h, new_w = [int(round(x * scale)) for x in (d, h, w)]

    # Resize (trilinear) - keeping HU values
    tensor = torch.tensor(volume).unsqueeze(0).unsqueeze(0)  # (1, 1, D, H, W)
    tensor_resized = F.interpolate(
        tensor, size=(new_d, new_h, new_w), mode='trilinear', align_corners=False
    )
    resized = tensor_resized.squeeze().cpu().numpy()

    # Center padding with 0 HU (water density)
    pad_d = target_d - new_d
    pad_h = target_h - new_h
    pad_w = target_w - new_w
    pad = (
        pad_w // 2, pad_w - pad_w // 2,
        pad_h // 2, pad_h - pad_h // 2,
        pad_d // 2, pad_d - pad_d // 2,
    )
    padded = np.pad(
        resized,
        ((pad[4], pad[5]), (pad[2], pad[3]), (pad[0], pad[1])),
        mode='constant',
        constant_values=-1000  
    )
    
    # Normalize to [-1, 1] then convert to [0, 255] for uint8
    normalized = padded / 1000.0  # [-1, 1]
    uint8_data = ((normalized + 1.0) / 2.0 * 255.0)  # Map [-1,1] to [0,255]
    uint8_data = np.clip(uint8_data, 0, 255)
    return uint8_data.astype(np.uint8)

In [3]:
path = "/Users/tianyuhan/Documents/data/ct-rate/dataset/train/train_1/train_1_a/train_1_a_1.nii.gz"
metadata = pd.read_csv("../data/train_metadata.csv")
volumename = os.path.basename(path)

metadata_row = metadata[metadata['VolumeName'] == volumename]
ct = read_and_preprocess_ct(path, {'RescaleSlope': 1,
                                   'RescaleIntercept': 0})
ct = ct.transpose(2, 1, 0)  # Transpose to (depth, height, width)
ct = resize_and_pad_3d_to_target(ct, target_shape=(160, 224, 224))
print(ct.shape)

(160, 224, 224)


In [5]:
def save_ct_to_mp4(ct, out_path='ct_movie.mp4', fps=15, window=None):
    """
    Save a 3D CT volume as an MP4 movie.
    
    Args:
        ct: numpy array of shape (D, W, H)
        out_path: output video file
        fps: frames per second
        window: (min, max) for intensity windowing (e.g., (0, 1024)). If None, auto window.
    """
    D, W, H = ct.shape
    
    # Window the image (normalize for display)
    if window is None:
        vmin, vmax = np.percentile(ct, 1), np.percentile(ct, 99)
    else:
        vmin, vmax = window

    # Setup video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(out_path, fourcc, fps, (H, W), isColor=False)

    for i in range(D):
        img = ct[i]  # Axial slice: shape (W, H)
        # Normalize to 0-255 for uint8 display
        # img = np.clip((img - vmin) / (vmax - vmin) * 255, 0, 255).astype(np.uint8)
        # img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # MP4 requires 3 channels
        video.write(img)

    video.release()
    print(f"Saved video: {out_path}")

In [6]:
save_ct_to_mp4(ct[:, ::-1, :], out_path='/Users/tianyuhan/Documents/data/inspect/ct_movie.mp4', fps=15)

Saved video: /Users/tianyuhan/Documents/data/inspect/ct_movie.mp4
